# Supply Chain tree for OpenX

In [ ]:
import validators
from collections import deque
import urllib.request, json 
import pandas as pd

sellers.json is a specification createtd to increase trust of the suplly chain. 

More information can be found at: https://iabtechlab.com/wp-content/uploads/2019/07/Sellers.json_Final.pdf

Sadly it is not that easy to get the suply chain, because not all comapnies share sellers.json file. And some companies share it, but not in proper way.

#### Problematic websites
https://www.ntvbmedia.com/sellers.json no domain key

https://didna.io/sellers.json - infinite lop

https://streamlyn.com/sellers.json - says that sidonews is INTERMEDIARY

https://www.sindonews.com/sellers.json - do not have any sellers

pmc.com -> https://ads.shemedia.com/sellers.json - no domain

https://adtelligent.com/sellers.json - domain starts with https://

Simple function that would allow us to traverse suply chainand store data as python dictionary would look like this. I would be a better approach to create class (structure) and then store information about variables in it. There are some limitations in this function. Every signle website would be accesed only once, so if there are multiple sellers sharing data at the same website we would need to give them the same "pointer".

In [ ]:
def get_nodes(url,urls=None):
    if urls is None:
        urls = []
    urls.append(url)
    if url.startswith('http://www.'):
        seller_url= 'http://' + url[len('http://www.'):]
    if url.startswith('www.'):
        seller_url= 'http://' + url[len('www.'):]
    if not url.startswith('http://'):
        seller_url= 'http://' + url
    else:
        seller_url = url
    if seller_url.endswith("/"):
        seller_url += "sellers.json"
    else:
        seller_url += "/sellers.json"

    try:
        with urllib.request.urlopen(seller_url) as url:
            # print(seller_url)
            data = json.loads(url.read().decode())
    except:
        return None
    if "sellers" in data.keys():
        for idx,elem in enumerate(data["sellers"]):
            if "seller_type" in elem.keys():
                if elem["seller_type"] in ["INTERMEDIARY", "BOTH"] and "domain" in elem.keys():
                    if validators.url(seller_url):
                        if (elem["domain"] not in urls):
                            data["sellers"][idx]["sellers_kids"] = get_nodes(elem["domain"],urls)
    return data

In [ ]:
# I have let this run for several hours, and it was always blocked at https://www.newsy.com/sellers.json
openx_url = "openx.com"
data = get_nodes (openx_url)

In [ ]:
print(data)

#### With this code we would be able to get the depth of suplly chain, assuming all comapnies are using sellers.json properly.

In [ ]:
def depth(d):
    queue = deque([(id(d), d, 1)])
    memo = set()
    while queue:
        id_, o, level = queue.popleft()
        if id_ in memo:
            continue
        memo.add(id_)
        if isinstance(o, dict):
            queue += ((id(v), v, level + 1) for v in o.values())
    return level

lvl = depth(data)
print(lvl)